## Modelado de los datos con Machine Learning

#### Importo las librerías que voy a necesitar en este proyecto

In [1]:
#Para trabajar con mis DataFrames y codificar numéricamente mis features
import pandas as pd

#Para separar en train y test, y para la validación cruzada
from sklearn.model_selection import train_test_split, KFold

#Para la regresión logística
from sklearn.linear_model import LogisticRegression

#Para el knn
from sklearn.neighbors import KNeighborsClassifier

#Para el svm de clasificación
from sklearn.svm import SVC

#Para el RandomForest de clasificación
from sklearn.ensemble import RandomForestClassifier

#Para la comparación de varios hiperparámetros
from sklearn.model_selection import GridSearchCV

#Para obtener la métrica recall
from sklearn.metrics import recall_score

#Para la comparación de varios modelos
from sklearn.pipeline import Pipeline

#Para números aleatorios
import numpy as np

#Fijo el máximo de filas a mostrar a 100 y el de columnas, a 50
pd.options.display.max_rows = 120
pd.options.display.max_columns = 50

### Importo el DataFrame que voy a utilizar

Es el df mushroom que he limpiado y acortado en el Notebook anterior, que contiene 11 features y un target dicotómicos.

In [2]:
df_mushroom =  pd.read_csv("../data/df_mushroom_corto_11_ft.csv")

df_mushroom

bruises_f  odor_f  odor_n  gill-size_b  gill-color_b  \
0         False   False   False        False         False   
1         False   False   False         True         False   
2         False   False   False         True         False   
3         False   False   False        False         False   
4          True   False    True         True         False   
...         ...     ...     ...          ...           ...   
8119       True   False    True         True         False   
8120       True   False    True         True         False   
8121       True   False    True         True         False   
8122       True   False   False        False          True   
8123       True   False    True         True         False   

      stalk-surface-above-ring_k  stalk-surface-above-ring_s  \
0                          False                        True   
1                          False                        True   
2                          False                        True   
3                          False                        True   
4                          False                        True   
...                          ...                         ...   
8119                       False                        True   
8120                       False                        True   
8121                       False                        True   
8122                       False                        True   
8123                       False                        True   

      stalk-surface-below-ring_k  ring-type_p  spore-print-color_h  \
0                          False         True                False   
1                          False         True                False   
2                          False         True                False   
3                          False         True                False   
4                          False        False                False   
...                          ...          ...                  ...   
8119                       False         True                False   
8120                       False         True                False   
8121                       False         True                False   
8122                        True        False                False   
8123                       False         True                False   

      population_v  class  
0            False      1  
1            False      0  
2            False      0  
3            False      1  
4            False      0  
...            ...    ...  
8119         False      0  
8120          True      0  
8121         False      0  
8122          True      1  
8123         False      0  

[8124 rows x 12 columns]

### División en train/test

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df_mushroom.drop('class', axis = 1), 
                                                    df_mushroom['class'], 
                                                    test_size = 0.2, 
                                                    random_state = 50) 
                                            #Mantengo la semilla de mi Notebook anterior

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (6499, 11)
X_test: (1625, 11)
y_train: (6499,)
y_test: (1625,)


### Tuneo los hiperparámetros de los modelos

#### 1) Buscando los mejores hiperparámetros de la regresión logística

Primero pruebo con penalties l2 y None, con lo cual los solver solo pueden ser lbfgs', 'newton-cg', ‘newton-cholesky’, ‘sag’ y ‘saga’.

In [4]:
#Defino el número de kfolds que quiero
kfold = KFold(n_splits = 10, shuffle = True, random_state = 26)  

#Instancio mi pipeline
pipe = Pipeline(steps=[('classifier', LogisticRegression())])

#Defino los parámetros de mis regresión logística
logistic_params = {'classifier': [LogisticRegression()],
                    'classifier__penalty': [None, 'l2'],
                    'classifier__C': np.arange(0.0001, 20, 2),
                    'classifier__solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']}

#Defino mi objeto GridSearchCV
log_reg_GSCV = GridSearchCV(estimator=pipe, param_grid=logistic_params, cv=kfold, verbose=4, n_jobs=-1, scoring="recall")

#Lo ajusto a mi datos de train
log_reg_GSCV.fit(X_train, y_train)

#Obtengo los resultados del GridSearchCV
results = log_reg_GSCV.cv_results_

#Imprimo los mejores resultados encontrados
print()
print("Mejor estimador:", log_reg_GSCV.best_estimator_.get_params()['classifier'])
print("Mejor combinación de hiperparámetros:", log_reg_GSCV.best_params_)
print("Mejor recall:", log_reg_GSCV.best_score_)
print()

#Imprimo el score para cada combinación de hiperparámetros
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f'Media de recall: {mean_score:.4f}, Hiperparámetros: {params}')

#En general veo que cualquier combinación de hiperparámetros consigue un recall de 0.9802, salvo aquella donde
    #el penalty es l2 y la C es baja.


Fitting 10 folds for each of 100 candidates, totalling 1000 fits

Mejor estimador: LogisticRegression(C=0.0001, penalty=None)
Mejor combinación de hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': None, 'classifier__solver': 'lbfgs'}
Mejor recall: 0.9801590646721798

Media de recall: 0.9802, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': None, 'classifier__solver': 'lbfgs'}
Media de recall: 0.9802, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': None, 'classifier__solver': 'newton-cg'}
Media de recall: 0.9802, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': None, 'classifier__solver': 'newton-cholesky'}
Media de recall: 0.9802, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': None, 'classifier__solver': 'sag'}
Media de rec

c:\Users\ellor\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Ahora pruebo con los penalty l1 y l2, y los solver compatibles.

In [5]:
#Defino el número de kfolds que quiero
kfold = KFold(n_splits = 10, shuffle = True, random_state = 26)  

#Instancio mi pipeline
pipe = Pipeline(steps=[('classifier', LogisticRegression())])

#Defino los parámetros de mis regresión logística
logistic_params_2 = {'classifier': [LogisticRegression()],
                    'classifier__penalty': ['l1', 'l2'],
                    'classifier__C': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1.0, 1.2, 1.5],
                    'classifier__solver': ['liblinear', 'saga']}

#Defino mi objeto GridSearchCV
log_reg_GSCV_2 = GridSearchCV(estimator=pipe, param_grid=logistic_params_2, cv=kfold, verbose=4, n_jobs=-1, scoring="recall")

#Lo ajusto a mi datos de train
log_reg_GSCV_2.fit(X_train, y_train)

#Obtengo los resultados del GridSearchCV
results_2 = log_reg_GSCV_2.cv_results_

#Imprimo los mejores resultados encontrados
print()
print("Mejor estimador:", log_reg_GSCV_2.best_estimator_.get_params()['classifier'])
print("Mejor combinación de hiperparámetros:", log_reg_GSCV_2.best_params_)
print("Mejor recall:", log_reg_GSCV_2.best_score_)
print()

#Imprimo el score para cada combinación de hiperparámetros
for mean_score, params in zip(results_2['mean_test_score'], results_2['params']):
    print(f'Media de recall: {mean_score:.4f}, Hiperparámetros: {params}')

#Al igual que antes, cualquier combinación de hiperparámetros consigue el recall máximo (0.9802), salvo cuando la C es baja.


Fitting 10 folds for each of 36 candidates, totalling 360 fits

Mejor estimador: LogisticRegression(C=0.01, penalty='l1', solver='saga')
Mejor combinación de hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.01, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Mejor recall: 0.9801590646721798

Media de recall: 0.0000, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Media de recall: 0.4000, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Media de recall: 0.8109, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}
Media de recall: 0.7835, Hiperparámetros: {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Media de r

#### 2) Buscando los mejores hiperparámetros del knn

In [6]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 66)  

pipe = Pipeline(steps=[('classifier', KNeighborsClassifier())])

knn_params = {'classifier': [KNeighborsClassifier()],
                'classifier__n_neighbors': [3, 8, 9, 10, 12, 15, 17, 19, 21, 25],
                'classifier__weights' : ["uniform", "distance"],
                'classifier__metric': ["euclidean", "manhattan"]}

knn_GSCV = GridSearchCV(estimator=pipe, param_grid=knn_params, cv=kfold, verbose=4, n_jobs=-1, scoring = "recall")

knn_GSCV.fit(X_train, y_train)

results = knn_GSCV.cv_results_

print()
print("Mejor estimador:", knn_GSCV.best_estimator_.get_params()['classifier'])
print("Mejor combinación de hiperparámetros:", knn_GSCV.best_params_)
print("Mejor recall:", knn_GSCV.best_score_)
print()

for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f'Media de recall: {mean_score:.4f}, Hiperparámetros: {params}')

#En el caso del knn, el recall es superior al de la regresión logística (0.9836), tanto con la 
    #métrica o norma euclídea como con la de manhattan, siempre y cuando 'classifier__weights' = 'distance'
    #y 'classifier__n_neighbors' >= 9.

Fitting 10 folds for each of 40 candidates, totalling 400 fits

Mejor estimador: KNeighborsClassifier(metric='euclidean', n_neighbors=9, weights='distance')
Mejor combinación de hiperparámetros: {'classifier': KNeighborsClassifier(), 'classifier__metric': 'euclidean', 'classifier__n_neighbors': 9, 'classifier__weights': 'distance'}
Mejor recall: 0.9835582307856828

Media de recall: 0.9750, Hiperparámetros: {'classifier': KNeighborsClassifier(), 'classifier__metric': 'euclidean', 'classifier__n_neighbors': 3, 'classifier__weights': 'uniform'}
Media de recall: 0.9750, Hiperparámetros: {'classifier': KNeighborsClassifier(), 'classifier__metric': 'euclidean', 'classifier__n_neighbors': 3, 'classifier__weights': 'distance'}
Media de recall: 0.9765, Hiperparámetros: {'classifier': KNeighborsClassifier(), 'classifier__metric': 'euclidean', 'classifier__n_neighbors': 8, 'classifier__weights': 'uniform'}
Media de recall: 0.9781, Hiperparámetros: {'classifier': KNeighborsClassifier(), 'classifie

#### 3) Buscando los mejores hiperparámetros del SVC

In [7]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 66)  

pipe = Pipeline(steps=[('classifier', SVC())])

svc_params = {'classifier': [SVC()],
                'classifier__kernel': ('linear', 'rbf', 'sigmoid'),
                'classifier__C': [0.7, 0.8, 0.9, 1, 1.5],
                'classifier__coef0': [-10, -1, 0, 0.1, 1, 10, 100],
                'classifier__gamma': ('scale', 'auto')}

svc_GSCV = GridSearchCV(estimator=pipe, param_grid=svc_params, cv=kfold, verbose=4, n_jobs=-1, scoring = "recall")

svc_GSCV.fit(X_train, y_train)

results = svc_GSCV.cv_results_

print()
print("Mejor estimador:", svc_GSCV.best_estimator_.get_params()['classifier'])
print("Mejor combinación de hiperparámetros:", svc_GSCV.best_params_)
print("Mejor recall:", svc_GSCV.best_score_)
print()

for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f'Media de recall: {mean_score:.4f}, Hiperparámetros: {params}')

#En el SVC encontramos un recall igual al del knn (0.9836), que se logra cuando 'classifier__C'>= 0.7, 
    #'classifier__gamma' = 'scale' y 'classifier__kernel' ='rbf'. 'classifier__coef0' no parece ser relevante


Fitting 10 folds for each of 210 candidates, totalling 2100 fits

Mejor estimador: SVC(C=0.8, coef0=-10)
Mejor combinación de hiperparámetros: {'classifier': SVC(), 'classifier__C': 0.8, 'classifier__coef0': -10, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Mejor recall: 0.9835582307856828

Media de recall: 0.9800, Hiperparámetros: {'classifier': SVC(), 'classifier__C': 0.7, 'classifier__coef0': -10, 'classifier__gamma': 'scale', 'classifier__kernel': 'linear'}
Media de recall: 0.9819, Hiperparámetros: {'classifier': SVC(), 'classifier__C': 0.7, 'classifier__coef0': -10, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Media de recall: 0.0000, Hiperparámetros: {'classifier': SVC(), 'classifier__C': 0.7, 'classifier__coef0': -10, 'classifier__gamma': 'scale', 'classifier__kernel': 'sigmoid'}
Media de recall: 0.9800, Hiperparámetros: {'classifier': SVC(), 'classifier__C': 0.7, 'classifier__coef0': -10, 'classifier__gamma': 'auto', 'classifier__kernel': 'linear'}
M

#### 4) Buscando los mejores hiperparámetros del RandomForestClassifier

In [8]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 66)  

pipe = Pipeline(steps=[('classifier', RandomForestClassifier())])

rfc_params = {'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [1, 25, 75, 150],
                'classifier__criterion': ('gini', 'entropy', 'log_loss'),
                'classifier__min_samples_split': [2, 5, 10, 30, 50],
                'classifier__min_samples_leaf': [1, 5, 10, 20],
                'classifier__max_features': ['sqrt', 'log2', None],
                'classifier__bootstrap': (True, False)}

rfc_GSCV = GridSearchCV(estimator=pipe, param_grid=rfc_params, cv=kfold, verbose=4, n_jobs=-1, scoring = "recall")

rfc_GSCV.fit(X_train, y_train)

results = rfc_GSCV.cv_results_

print()
print("Mejor estimador:", rfc_GSCV.best_estimator_.get_params()['classifier'])
print("Mejor combinación de hiperparámetros:", rfc_GSCV.best_params_)
print("Mejor recall:", rfc_GSCV.best_score_)
print()

for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f'Media de recall: {mean_score:.4f}, Hiperparámetros: {params}')

#Como en el knn y el SVC, el recall máximo alcanzado es 0.9836.
#En términos generales, criterion, max_features y n_estimators no parecen ser 
    #hiperparámetros determinantes con estos datos. 
    #Lo mejor es min_samples_leaf = 1, y min_samples_split < 30

Fitting 10 folds for each of 1440 candidates, totalling 14400 fits

Mejor estimador: RandomForestClassifier(n_estimators=1)
Mejor combinación de hiperparámetros: {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__criterion': 'gini', 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 1}
Mejor recall: 0.9835582307856828

Media de recall: 0.9836, Hiperparámetros: {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__criterion': 'gini', 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 1}
Media de recall: 0.9836, Hiperparámetros: {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__criterion': 'gini', 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimat

In [9]:
#Como parece ser que criterion, max_features y n_estimators no son hiperparámetros 
#determinantes, los elimino de mi GridSearh y sigo probando combinaciones 
#de hiperparámetros para ver si mejora el recall

kfold = KFold(n_splits = 10, shuffle = True, random_state = 66)  

pipe = Pipeline(steps=[('classifier', RandomForestClassifier())])

rfc_params_2 = {'classifier': [RandomForestClassifier()],
                'classifier__min_samples_split': [2, 4, 6, 8, 10, 12, 15, 30, 35, 40],
                'classifier__min_samples_leaf': [1, 2, 3, 4, 5],
                'classifier__max_depth': [1, 3, 5, 6, 7, 8, None],
                'classifier__class_weight': ('balanced', 'balanced_subsample'),
                'classifier__bootstrap': (True, False)}

rfc_GSCV_2 = GridSearchCV(estimator=pipe, param_grid=rfc_params_2, cv=kfold, verbose=4, n_jobs=-1, scoring = "recall")

rfc_GSCV_2.fit(X_train, y_train)

results_2 = rfc_GSCV_2.cv_results_

print()
print("Mejor estimador:", rfc_GSCV_2.best_estimator_.get_params()['classifier'])
print("Mejor combinación de hiperparámetros:", rfc_GSCV_2.best_params_)
print("Mejor recall:", rfc_GSCV_2.best_score_)
print()

for mean_score, params in zip(results_2['mean_test_score'], results_2['params']):
    print(f'Media de recall: {mean_score:.4f}, Hiperparámetros: {params}')

#En este caso parece ser que class_weigth tampoco parece afectar, y que lo más
    #determinante es que la profundidad máxima sea de al menos 6, 
    #o no tenga profuncidad máxima

#En cualquier caso, el recall máximo alcanzado es el mismo que en los casos
    #anteriores: 0.9835582307856828

Fitting 10 folds for each of 1400 candidates, totalling 14000 fits

Mejor estimador: RandomForestClassifier(class_weight='balanced', max_depth=6)
Mejor combinación de hiperparámetros: {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__class_weight': 'balanced', 'classifier__max_depth': 6, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}
Mejor recall: 0.9835582307856828

Media de recall: 0.9143, Hiperparámetros: {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__class_weight': 'balanced', 'classifier__max_depth': 1, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}
Media de recall: 0.9177, Hiperparámetros: {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__class_weight': 'balanced', 'classifier__max_depth': 1, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 4}
Media de recall: 0.9364, Hiperparámetros: {'classifier': RandomFo

#### Tabla resumen del recall y los hiperparámetros en los cuatro modelos, en el GridSearchCV


In [10]:
pd.set_option('display.max_colwidth', 250)

modelos = ['Logistic Regression (1)', 'Logistic Regression (1)', 'knn', 'SVC', 'Random Forest Classifier (1)', 'Random Forest Classifier (2)']
mejor_recall = [log_reg_GSCV.best_score_, log_reg_GSCV_2.best_score_, knn_GSCV.best_score_, svc_GSCV.best_score_, rfc_GSCV.best_score_, rfc_GSCV_2.best_score_]
mejor_hiperparam = [log_reg_GSCV.best_params_, log_reg_GSCV_2.best_params_, knn_GSCV.best_params_, svc_GSCV.best_params_, rfc_GSCV.best_params_, rfc_GSCV_2.best_params_]

compara_modelos = pd.DataFrame({'Modelo': modelos, 'Recall más alto': mejor_recall,
                              'Mejores hiperparámetros': mejor_hiperparam}).set_index('Modelo')

compara_modelos

Recall más alto  \
Modelo                                          
Logistic Regression (1)              0.980159   
Logistic Regression (1)              0.980159   
knn                                  0.983558   
SVC                                  0.983558   
Random Forest Classifier (1)         0.983558   
Random Forest Classifier (2)         0.983558   

                                                                                                                                                                                                                                                         Mejores hiperparámetros  
Modelo                                                                                                                                                                                                                                                                            
Logistic Regression (1)                                                                                                                                {'classifier': LogisticRegression(), 'classifier__C': 0.0001, 'classifier__penalty': None, 'classifier__solver': 'lbfgs'}  
Logistic Regression (1)                                                                                                                                   {'classifier': LogisticRegression(), 'classifier__C': 0.01, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}  
knn                                                                                                                                   {'classifier': KNeighborsClassifier(), 'classifier__metric': 'euclidean', 'classifier__n_neighbors': 9, 'classifier__weights': 'distance'}  
SVC                                                                                                                                             {'classifier': SVC(), 'classifier__C': 0.8, 'classifier__coef0': -10, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}  
Random Forest Classifier (1)  {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__criterion': 'gini', 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 1}  
Random Forest Classifier (2)                                  {'classifier': RandomForestClassifier(), 'classifier__bootstrap': True, 'classifier__class_weight': 'balanced', 'classifier__max_depth': 6, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}

Al comparar el desempeño de los cuatro modelos anteriores, vemos que **tanto el knn, como el SVC y el RandomForestClassifier logran clasificar** adecuadamente a las setas comestibles vs venenosas, **con un 98.36% de sensibilidad**.

Para nuestro proyecto, tomaremos como modelo a utilizar el RandomForestClassifier, que suele ser el más robusto, y clasificamos los datos de test.

In [11]:
#Dejo la combinación de valores por defecto, ya que he visto en mis GridSearchCV
    #anteriores que es capaz de clasificar con el máximo recall alcanzado

random_forest = RandomForestClassifier()                                       
random_forest.fit(X_train, y_train)

y_pred_train = random_forest.predict(X_train)
random_forest_score_train = recall_score(y_train, y_pred_train)
print("recall en train:", random_forest_score_train)

y_pred_test = random_forest.predict(X_test)
random_forest_score_test = recall_score(y_test, y_pred_test)
print("recall en test:", random_forest_score_test)

recall en train: 0.9835164835164835
recall en test: 0.9744525547445255


**Con solo 11 features, mi modelo es capaz de clasificar correctamente las setas en test con un 97.44% de sensibilidad**.